In [1]:
from tensorflow.keras.datasets import imdb
(X_train,y_train),(X_test,y_test) = imdb.load_data()

17473536/17464789 [==============================] - 0s 0us/step


In [2]:
max(len(s)for s in X_train), sum(map(len,X_train))/len(X_train)

(2494, 238.71364)

In [3]:
import tensorflow as tf
import numpy as np
seed=2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [13]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [5]:
(X_train,y_train),(_, _)=imdb.load_data(num_words=10000)

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(
    X_train,y_train,stratify=y_train, test_size=0.2,random_state=seed
)

In [7]:
max_len=500
num_words=10000
X_train=pad_sequences(X_train,maxlen=max_len)
X_test=pad_sequences(X_test,maxlen=max_len)


In [8]:
model=Sequential ([
                   Embedding(num_words,100,input_length=max_len),
                   LSTM(128), #activation='tanh'는 디폴트.
                   Dense(1,activation='sigmoid')
                   ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile('adam','binary_crossentropy',['accuracy'])

In [10]:
model_path='best-imdb-lstm.h5'
mc=ModelCheckpoint(model_path,verbose=1,save_best_only=True)
es=EarlyStopping(patience=5)

In [11]:
hist=model.fit(
    X_train,y_train,validation_split=0.2,
    epochs=30,batch_size=64,callbacks=[mc,es]
)

Epoch 1/30
249/250 [============================>.] - ETA: 0s - loss: 0.5222 - accuracy: 0.7334
Epoch 1: val_loss improved from inf to 0.39719, saving model to best-imdb-lstm.h5
250/250 [==============================] - 17s 40ms/step - loss: 0.5217 - accuracy: 0.7337 - val_loss: 0.3972 - val_accuracy: 0.8257
Epoch 2/30
249/250 [============================>.] - ETA: 0s - loss: 0.3148 - accuracy: 0.8730
Epoch 2: val_loss improved from 0.39719 to 0.30904, saving model to best-imdb-lstm.h5
250/250 [==============================] - 9s 37ms/step - loss: 0.3158 - accuracy: 0.8724 - val_loss: 0.3090 - val_accuracy: 0.8730
Epoch 3/30
249/250 [============================>.] - ETA: 0s - loss: 0.2121 - accuracy: 0.9179
Epoch 3: val_loss did not improve from 0.30904
250/250 [==============================] - 9s 36ms/step - loss: 0.2121 - accuracy: 0.9180 - val_loss: 0.3428 - val_accuracy: 0.8615
Epoch 4/30
249/250 [============================>.] - ETA: 0s - loss: 0.1367 - accuracy: 0.9516
Epoc

In [15]:
best_model=load_model(model_path)
best_model.evaluate(X_test,y_test)

157/157 [==============================] - 2s 11ms/step - loss: 0.3261 - accuracy: 0.8706


[0.32609841227531433, 0.8705999851226807]